In [1]:
from spacepy.pycdf import lib as cdflib
from datetime import datetime, timedelta
import pytz
import os
os.environ["TZ"]="UTC"

In [2]:
start_time = datetime(1950,1,1, tzinfo=pytz.UTC)
def entry(dt):
    tt2000=cdflib.datetime_to_tt2000(dt)
    epoch=cdflib.datetime_to_epoch(dt)
    epoch16=cdflib.datetime_to_epoch16(dt)
    entry = f"""
    //{dt}
    test_entry{{ {int(dt.timestamp())}, {{ {int(tt2000)} }}, {{ {epoch} }}, {{ {epoch16[0]}, {epoch16[1]} }} }},
"""
    entry
    return entry

In [3]:
t = start_time
table = """

#include "chrono/cdf-chrono.hpp"
#include <array>
#include <cstdint>

struct test_entry
{
    int64_t        unix_epoch;
    cdf::tt2000_t  tt2000_epoch;
    cdf::epoch     epoch_epoch;
    cdf::epoch16   epoch16_epoch;
    
};

test_entry test_values[] = {
"""
while t < datetime(2100,1,1, tzinfo=pytz.UTC):
    table += entry(t)
    t+=timedelta(days=100)
    
for y in range(1972,2020):
    for m in [1,7]:
        for dt in range(-2,2):
            table += entry(datetime(y,m,1, tzinfo=pytz.UTC)+timedelta(seconds=dt))
    
table += "};"

In [4]:
with open("../tests/chrono/test_values.hpp", "w") as f:
    f.write(table)